In [1]:
# importing libs
import numpy as np
import tensorflow as tf
import keras
from keras.layers import Input, Dense, GaussianNoise,Lambda,Dropout
from keras.models import Model
from keras.models import Sequential
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam,SGD
from keras import backend as K

Using TensorFlow backend.


In [2]:
# the first stage training of the two-stage training autoencoder for the relay network
# S-R link traing, to determine the source encoding and relay decoding
# for n = 8

In [3]:
# for reproducing reslut
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(3)

In [4]:
# defining parameters
# define (n,k) here for (n,k) autoencoder
# n = n_channel 
# k = log2(M)  ==> so for (7,4) autoencoder n_channel = 7 and M = 2^4 = 16 
M = 2**8
k = np.log2(M)
k = int(k)
n_channel = 8
R = k/n_channel
print ('M:',M,'k:',k,'n:',n_channel,'R:',R)

M: 256 k: 8 n: 8 R: 1.0


In [5]:
#generating data of size N
N = 8000
label = np.random.randint(M,size=N)

In [6]:
# creating one hot encoded vectors
data = []
for i in label:
    temp = np.zeros(M)
    temp[i] = 1
    data.append(temp)

In [7]:
# checking data shape
data = np.array(data)
print (data.shape)

(8000, 256)


In [8]:
# checking generated data with it's label
temp_check = [17,23,45,67,89,96,72,250,350]
for i in temp_check:
    print(label[i],data[i])

25 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
212 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

In [9]:
def frange(x, y, jump):
  while x < y:
    yield x
    x += jump

In [10]:
# defining autoencoder and it's layer
input_signal = Input(shape=(M,))
encoded = Dense(M, activation='tanh')(input_signal) 
encoded1 = Dense(n_channel, activation='linear')(encoded)

def mynormalization(x):
  from keras import backend as K
  return np.sqrt(n_channel)*K.l2_normalize(x,axis=1) 
# Energy per channel use will be 1 (unit power), fair comparison with PSK

encoded2 = Lambda(mynormalization)(encoded1)
# per-channel noise variance, same as per-bit for R=1
encoded3 = GaussianNoise(np.sqrt(1/(2*R*10.0**(np.random.randint(-2, 8.5) /10.0))))(encoded2)
# relay detection
decoded = Dense(M, activation='linear')(encoded3) # 
decoded1 = Dense(M, activation='softmax')(decoded) # 

autoencoder_SR = Model(input_signal, decoded1)

In [11]:
adam = Adam(lr=0.001)
autoencoder_SR.compile(optimizer=adam, loss='categorical_crossentropy')

In [12]:
# printing summary of layers and it's trainable parameters 
print (autoencoder_SR.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 2056      
_________________________________________________________________
lambda_1 (Lambda)            (None, 8)                 0         
_________________________________________________________________
gaussian_noise_1 (GaussianNo (None, 8)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               2304      
_________________________________________________________________
dense_4 (Dense)              (None, 256)               65792     
Total para

In [13]:
# for tensor board visualization
#tbCallBack = keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32, write_graph=True, write_grads=True, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)

In [14]:
# traning auto encoder
hist=autoencoder_SR.fit(data, data,
                epochs=80,
                batch_size=50)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/80





8000/8000 [==============================] - 5s 656us/step - loss: 4.1804
Epoch 2/80
8000/8000 [==============================] - 2s 304us/step - loss: 2.7336
Epoch 3/80
8000/8000 [==============================] - 3s 328us/step - loss: 2.3053
Epoch 4/80
8000/8000 [==============================] - 3s 387us/step - loss: 2.1666
Epoch 5/80
8000/8000 [==============================] - 3s 326us/step - loss: 2.1372
Epoch 6/80
8000/8000 [==============================] - 2s 312us/step - loss: 2.0525
Epoch 7/80
8000/8000 [==============================] - 3s 336us/step - loss: 2.0828
Epoch 8/80
8000/8000 [==============================] - 3s 324us/step - loss: 2.0363
Epoch 9/80
8000/8000 [==============================] - 2s 301us/step - loss: 2.0576
Epoch 10/80
8000/8000 [==============================] - 3s 331us/step - loss: 2.0429
Epoch 11/80
8000/8000 [========================

Epoch 73/80
8000/8000 [==============================] - 3s 351us/step - loss: 1.9310
Epoch 74/80
8000/8000 [==============================] - 3s 319us/step - loss: 1.9449
Epoch 75/80
8000/8000 [==============================] - 2s 302us/step - loss: 1.9362
Epoch 76/80
8000/8000 [==============================] - 3s 328us/step - loss: 1.9818
Epoch 77/80
8000/8000 [==============================] - 2s 302us/step - loss: 1.9683
Epoch 78/80
8000/8000 [==============================] - 2s 312us/step - loss: 1.9470
Epoch 79/80
8000/8000 [==============================] - 2s 305us/step - loss: 1.9772
Epoch 80/80
8000/8000 [==============================] - 3s 326us/step - loss: 1.9651


In [15]:
# # print the layers and weights
# for lay in autoencoder_SR.layers:
#     print(lay.name)
#     print(lay.get_weights())
# print(hist.history.keys())

input_1
[]
dense_1
[array([[-0.03485795, -0.09312075, -0.01856875, ...,  0.0477182 ,
         0.07682346,  0.10623781],
       [-0.03797137,  0.11708888, -0.10711638, ..., -0.03618057,
         0.3061855 , -0.13295652],
       [ 0.04183083,  0.07549759, -0.2165226 , ...,  0.10170979,
         0.23636386, -0.10616776],
       ...,
       [-0.17220442,  0.13694054, -0.17908585, ...,  0.15658356,
         0.31790265,  0.03979917],
       [-0.20103617,  0.03800648,  0.13624527, ..., -0.19704346,
         0.06989356, -0.0893888 ],
       [ 0.02975768, -0.08755005, -0.00066905, ..., -0.03568891,
        -0.19141604,  0.09947579]], dtype=float32), array([ 1.2506972e-04, -6.6676536e-03,  6.7121414e-03, -1.6147555e-03,
        4.3270686e-03, -2.6672166e-03,  1.9608501e-03,  3.4822901e-03,
       -2.3381942e-04,  5.9381016e-03, -4.4254164e-04, -1.1828599e-03,
       -6.4034276e-03, -4.2287363e-03,  5.3620618e-03, -9.6915208e-04,
        3.0080631e-04, -6.0701845e-03, -8.9952105e-04,  1.0770720e-

[array([[-0.1363531 ,  0.13229367, -0.07839827, ..., -0.01097638,
         0.15951377, -0.07576041],
       [ 0.06030845,  0.01827411,  0.02529485, ..., -0.02777753,
        -0.1418681 ,  0.05839665],
       [ 0.0020199 ,  0.00243485,  0.17763181, ...,  0.13269949,
        -0.04966784, -0.02599123],
       ...,
       [-0.03178185, -0.0424644 , -0.19944906, ..., -0.0686636 ,
         0.1331853 , -0.0902095 ],
       [ 0.09154726, -0.00455543,  0.05101276, ..., -0.13848518,
        -0.1880211 ,  0.12054257],
       [ 0.06484296, -0.1765913 ,  0.07128783, ..., -0.04641288,
        -0.04211171, -0.0888099 ]], dtype=float32), array([-4.18724827e-02, -1.47551205e-02,  5.93607798e-02, -3.79062742e-02,
       -6.01266213e-02, -2.88586691e-02, -9.05904267e-03,  3.90001386e-02,
       -6.86919456e-03,  3.35555375e-02, -5.12454547e-02, -4.10143882e-02,
       -1.61296502e-02, -6.14574999e-02,  1.50016753e-03,  4.14353870e-02,
       -7.32615292e-02, -1.22748323e-01, -1.13528315e-02,  2.33945902e

In [20]:
# making encoder from full autoencoder
encoder_SR = Model(input_signal, encoded2)

In [21]:
# making decoder from full autoencoder
encoded_input = Input(shape=(n_channel,))

# 3: decoder should also be modified
# deco = autoencoder.layers[-3](encoded_input)
deco = autoencoder_SR.layers[-2](encoded_input)
deco_1 = autoencoder_SR.layers[-1](deco)
decoder_SR = Model(encoded_input, deco_1)

In [18]:
# saving autoencoder_SR
# HDF5 file, you have to pip3 install h5py if don't have it
# if you want to save model then remove below comment
# import h5py
# autoencoder_SR.save('autoencoder_SR.h5')
# encoder_SR.save('encoder_SR.h5')
# decoder_SR.save('decoder_SR.h5')

In [19]:
# checking the S-R link BLER

In [22]:
# generating data for checking BLER
N = 1*10**5
test_label = np.random.randint(M,size=N)
test_data = []

for i in test_label:
    temp = np.zeros(M)
    temp[i] = 1
    test_data.append(temp)
    
test_data = np.array(test_data)

In [23]:
# calculating BLER
EbNodB_range = list(frange(-2,8.5,2))
# EbNodB_range = list(frange(8,8.5,1))
bler = [None]*len(EbNodB_range)
for n in range(0,len(EbNodB_range)):
    EbNo=10.0**(EbNodB_range[n]/10.0)
    noise_std = np.sqrt(1/(2*R*EbNo))
    noise_mean = 0
    no_errors = 0
    nn = N
    noise = noise_std * np.random.randn(nn,n_channel)
    encoded_signal = encoder_SR.predict(test_data) 
    final_signal = encoded_signal + noise
    pred_final_signal =  decoder_SR.predict(final_signal)
    pred_output = np.argmax(pred_final_signal,axis=1)
    no_errors = (pred_output != test_label)
    no_errors =  no_errors.astype(int).sum()
    bler[n] = no_errors / nn 
    print ('SNR:',EbNodB_range[n],'BLER:',bler[n])
    # use below line for generating matlab like matrix which can be copy and paste for plotting ber graph in matlab
    # print(bler[n], " ",end='')

SNR: -2 BLER: 0.64052
SNR: 0 BLER: 0.43181
SNR: 2 BLER: 0.21372
SNR: 4 BLER: 0.06498
SNR: 6 BLER: 0.01087
SNR: 8 BLER: 0.00089


In [ ]:
# ploting ber curve
# save(ber, 'BLER_aotuencoder')
np.savetxt('BLER-SRlink-AE-ROB.txt', bler)
import matplotlib.pyplot as plt
from scipy import interpolate
plt.plot(EbNodB_range, bler, 'bo',label='Autoencoder-SRlink')
plt.yscale('log')
plt.xlabel('SNR (dB)')
plt.ylabel('BLER for S-R link')
plt.grid()
plt.legend(loc='upper right',ncol = 1)